In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import os
import sys
import matplotlib.pyplot as plt
import pickle
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import HTML
import matplotlib.animation as animation
from moviepy.editor import VideoClip, ImageSequenceClip
from moviepy.video.io.bindings import mplfig_to_npimage
import torch
import sys
sys.path.append('../src')
sys.path.append('../DanceRevolution')
sys.path.append('../DanceRevolution/v2')
# from director import Director
import yaml
import skeleton_sequence
from skeleton_sequence import SkeletonSequence
# from utilities import get_best_results_df, extract_sub_dict
from tqdm import tqdm

DX: compare function is used to compare two sequences of skeletons actions, which in my case are sequences interpolated by linear and bezier methods.

In [2]:
def compare(s, smooth_net, degree, least_square=None, duration=60, nrows=1, ncols=2, ax_limits=None, add_least_square=False, figsize=None):
    assert not (add_least_square and least_square is None)
    
    to_compare = [least_square, smooth_net] if add_least_square else [smooth_net]
    titles = [f'Least-square d={degree}', f'SmoothNet d={degree}'] if add_least_square else [f'SmoothNet d={degree}']
    
    animation = s.animate_skeleton(xyz=to_compare, body=0, comparison=True, comparison_title=titles, duration=duration, 
                                   nrows=nrows, ncols=ncols, ax_limits=ax_limits, fps=None, figsize=figsize)
    return animation

In [ ]:
def get_output(director, rational=True):
    out = []
    
    with torch.no_grad():
        for input_tuple in tqdm(director.test_data_loader, desc='Working'):
            input_tensor, labels, metadata = director.tester.split_input_tuple(input_tuple)
            input_tensor = director.tester.tensor_to_cuda(input_tensor)
            labels = director.tester.tensor_to_cuda(labels)
            output, _ = director.tester._get_output(input_tensor, labels, metadata)
                                                
            cp, raw_, raw, bez = output['cp'], output['raw_'], output['raw'], output['bez']
            B, C, T, V, M = raw.shape
            w = output['w'] if rational else None
            
            smoothed_skeleton, cp = director.tester.get_smoothed_skeleton(raw_, cp, w)
            smoothed_skeleton = smoothed_skeleton.reshape(B, M, C, T, V).permute(0, 2, 3, 4, 1)
            
            for i in range(B):
                met = extract_sub_dict(metadata, i, keys=('uid', 'label_index', 'label_str', 'index'))
                raw_sequence = director.test_dataset.holder.skeletons[met['index']]
                least_square_seq = bez[i].cpu().numpy()
                smooth_net_seq = smoothed_skeleton[i].cpu().numpy()

                out.append({'raw_seq': raw_sequence, 'bez': least_square_seq, 'smooth_net': smooth_net_seq, 'cp': cp[i]})

    return out

In [3]:
p = '/mnt/lustre/dmoltisanti/data/experiments/dance/dance-revolution/smooth-net/reconstruct/raw+bcurve/opt=adam-loss=l1-lr=0.0001-lrs=multistep_lr-bs=64-hidden_layers=[512, 512, 512, 512, 512, 512, 512, 512]-bez_degree=50-sliding_w=60-w_overlap=20'

with open(p + '/smooth-net-xview.yaml') as f:
    settings = yaml.load(f, Loader=yaml.FullLoader)
    director = Director(settings, test_only=True, checkpoint_path=p + '/checkpoints/best_test_min_loss.pt')
    out = get_output(director, rational=True)

FileNotFoundError: [Errno 2] No such file or directory: '/mnt/lustre/dmoltisanti/data/experiments/dance/dance-revolution/smooth-net/reconstruct/raw+bcurve/opt=adam-loss=l1-lr=0.0001-lrs=multistep_lr-bs=64-hidden_layers=[512, 512, 512, 512, 512, 512, 512, 512]-bez_degree=50-sliding_w=60-w_overlap=20/smooth-net-xview.yaml'

So after I get the bezier interpolated, I can compare them here.

In [4]:
from dataset_holder import DanceRevolutionHolder
from dataset import DanceRevolutionDataset
holder = DanceRevolutionHolder('/home/dingxi/DanceRevolution/data/test_1min', 'test')
dataset = DanceRevolutionDataset(holder, data_in='raw+bcurve')

In [5]:
dataset[1]

TypeError: unsupported operand type(s) for +: 'NoneType' and 'int'

In [ ]:
d = out[0]
print('uid', d['raw_seq'].metadata['uid'])
animation = compare(d['raw_seq'], d['smooth_net'], settings['dataset_obj']['bez_degree'], ax_limits=(-1, 1, -1, 1), figsize=(10, 3))
animation.ipython_display(loop=True, autoplay=True, maxduration=60, fps=30)